In [1]:
# imports
# import the necessary packages
from sklearn.linear_model import LogisticRegression
from keras.applications import ResNet50
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications.resnet50 import preprocess_input, decode_predictions
from config import dogs_vs_cats_config as config
from imutils import paths
import numpy as np
from collections import namedtuple
import pickle
import os

Using TensorFlow backend.


In [2]:
def load_model(model_path = config.LOGISTIC_TRAINED_MODEL):
    ''' Load the logistic model and return it '''
    with open(model_path, 'rb') as infh:
        model = pickle.load(infh)
    return model

In [3]:
def load_test_images_paths(test_image_path = config.TEST_COMPETITION_PATH):
    ''' Get the paths of the test images '''
    test_image_paths = list(paths.list_images(test_image_path))
    return test_image_paths

In [4]:
def  image_thru_resnet50_xfer(res_model, image_path):
    ''' Do the preprocessing step of passing the image through the ResNet50 lower level
        nets (and not the final FC layers)
        That is, process the image exactly as it would before being fed to the logistic classifier
        during the training step'''
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = imagenet_utils.preprocess_input(image)
    # Now pass the image through the res_model "predict". This will not predict the result, but will
    # instead pass it through the conv layers onel of resnet50 (and not the FC layers).
    # meaning, res_image will be the extracted "features" of the image (remember, we are doing xfer learning here)
    res_image = res_model.predict(image)
    return res_image

In [7]:
def predict_test_set():
    ''' Go through all of the test images and predict '''
    # Get the output in CSV format
    img_pred = namedtuple('img_pred', 'id label')
    header = img_pred('id', 'label')
    submission = [header]
    
    # Load the trained model
    logistic_model = load_model()
    # Load the resnet50 model
    res_model = ResNet50(weights="imagenet", include_top=False)
    # Get the test images paths
    test_image_paths = load_test_images_paths()
    # test_image_paths = test_image_paths[0:10]
    
    for image_path in test_image_paths:
        res_image = image_thru_resnet50_xfer(res_model, image_path)
        prediction = logistic_model.predict_proba(res_image.reshape(1,-1))
        image_fn = os.path.basename(image_path)
        image_id = os.path.splitext(image_fn)[0]
        image_pred = prediction[0][1]  # (second) Index 0 => Cat and Index 1 => Dog
        formatted_pred = '{:f}'.format(float(image_pred))
        submission.append(img_pred(image_id, formatted_pred))
    return submission

In [6]:
submission = predict_test_set()

/home/ggopalan/anaconda3/envs/ggpy3/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [9]:
with open('gg_catsvdogs_sub_01.csv', 'wt') as outfh:
    for (id, label) in submission:
        outfh.write('{},{}\n'.format(id, label))